# colab setting, read file and import module

In [1]:
#!pip install google.colab #如未安裝取消註解後執行
import os
from google.colab import drive
drive.mount('/content/my_drive')

Mounted at /content/my_drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
# file_path = '/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/xgboost_month_freq_dataset.csv'
# df = pd.read_csv(file_path)

# 又是這裡....主表與591表格靠攏


In [3]:
need_df = pd.read_csv('/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/full_main_data_low_var_v2.csv', encoding='utf-8')

<ipython-input-3-4a48e88a98d4>:1: DtypeWarning: Columns (31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  need_df = pd.read_csv('/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/full_main_data_low_var_v2.csv', encoding='utf-8')


In [5]:
need_df.info(max_cols=114)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 783397 entries, 0 to 783396
Data columns (total 114 columns):
 #    Column                                Non-Null Count   Dtype  
---   ------                                --------------   -----  
 0    usage_category                        758618 non-null  object 
 1    address                               783397 non-null  object 
 2    property_type                         783397 non-null  object 
 3    community_name                        783397 non-null  int64  
 4    main_building_ratio                   746392 non-null  float64
 5    city                                  783397 non-null  object 
 6    parking_space_price                   183395 non-null  float64
 7    transaction_date                      783397 non-null  object 
 8    elevator_available                    783397 non-null  int64  
 9    building_age                          595528 non-null  float64
 10   number_of_land                        783397 non-null 

In [6]:
# 丟經濟指標
need_df.drop(need_df.columns[33:74], axis=1, inplace=True)

In [7]:
need_df.info(max_cols=114)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 783397 entries, 0 to 783396
Data columns (total 73 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   usage_category                        758618 non-null  object 
 1   address                               783397 non-null  object 
 2   property_type                         783397 non-null  object 
 3   community_name                        783397 non-null  int64  
 4   main_building_ratio                   746392 non-null  float64
 5   city                                  783397 non-null  object 
 6   parking_space_price                   183395 non-null  float64
 7   transaction_date                      783397 non-null  object 
 8   elevator_available                    783397 non-null  int64  
 9   building_age                          595528 non-null  float64
 10  number_of_land                        783397 non-null  int64  
 11  

# 基礎捕值fillna

## main_building_ratio
倉庫、工廠:、其他:、廠辦:、農舍:、辦公商業大樓:、透天厝:
這幾個幾乎全部都是nan
其中[倉庫, 工廠, 農舍, 辦公商業大樓, 透天厝]可以假設是"土地類的"，所以可以給100%
"其他"跟"廠辦"就比較難定義，那我覺得可以補全體的平均值

In [8]:
fillna_df = need_df.copy()

In [9]:
fillna_df['property_type'].isnull().sum()

0

In [10]:
fillna_df['property_type'].unique()

array(['公寓(5樓含以下無電梯)', '華廈(10層含以下有電梯)', '套房(1房(1廳)1衛)', '住宅大樓(11層含以上有電梯)',
       '店面（店舖)', '透天厝', '其他'], dtype=object)

In [11]:
fillna_df['main_building_ratio'].isnull().sum()

37005

In [ ]:
# fillna_df[fillna_df['property_type'] == '透天厝']['main_building_ratio'].info()

In [12]:
fillna_df.loc[fillna_df['property_type'] == '透天厝', 'main_building_ratio'] = 100

# 又一次叫你不要寫迴圈....
# for index, value in enumerate(fillna_df['property_type']):
#   if value ==  '透天厝':
#     fillna_df.loc[index,['main_building_ratio']] = 100

In [13]:
fillna_df['main_building_ratio'].isnull().sum()

14499

In [14]:
fillna_df['property_type'].isnull().sum()

0

In [15]:
type_p_list = fillna_df['property_type'].unique()
type_p_list

array(['公寓(5樓含以下無電梯)', '華廈(10層含以下有電梯)', '套房(1房(1廳)1衛)', '住宅大樓(11層含以上有電梯)',
       '店面（店舖)', '透天厝', '其他'], dtype=object)

In [16]:
type_p_list_2 = list(type_p_list)

In [17]:
# 其他要最後填，先排掉
type_p_list_2.pop(6)

'其他'

In [18]:
# 我們可以直接使用 .loc 來改變原本的 DataFrame，然後將平均值指派給那些需要填補的索引處。改寫如下：，今天已經兩次這個問題了....

for type_p in type_p_list_2:
  tmp = fillna_df[fillna_df['property_type'] == type_p ]['main_building_ratio']
  need_fillna_index = tmp[ tmp.isnull() ].index

  # 錯誤示範: 簡單來說只要是切片索引下就不能就用fillna直接改變dataframe
  # fillna_df.loc[need_fillna_index, 'main_building_ratio'].fillna(tmp.mean(), inplace=True)

  # 正確反正我們已經找到特定property_type下na的index了，在原先的data_clean使用loc只需要在指定欄位main_building_ratio，我們就可以直接進行填值(mean)
  mean_value = tmp.mean()
  fillna_df.loc[need_fillna_index, 'main_building_ratio'] = mean_value

In [19]:
type_p_list = fillna_df['property_type'].unique()
for type_p in type_p_list:
  print(f"{type_p}:")
  tmp = fillna_df[fillna_df['property_type'] == type_p ]['main_building_ratio']
  print(f"{type_p} mean ratio:{tmp.mean()}")
  print(f"{type_p}'s main_building_ratio null ratio is {tmp.isnull().sum()}/{len(tmp)}")
  print('-'*50)

公寓(5樓含以下無電梯):
公寓(5樓含以下無電梯) mean ratio:90.61000159863887
公寓(5樓含以下無電梯)'s main_building_ratio null ratio is 0/179542
--------------------------------------------------
華廈(10層含以下有電梯):
華廈(10層含以下有電梯) mean ratio:61.14878857359721
華廈(10層含以下有電梯)'s main_building_ratio null ratio is 0/128553
--------------------------------------------------
套房(1房(1廳)1衛):
套房(1房(1廳)1衛) mean ratio:60.6382584564407
套房(1房(1廳)1衛)'s main_building_ratio null ratio is 0/36627
--------------------------------------------------
住宅大樓(11層含以上有電梯):
住宅大樓(11層含以上有電梯) mean ratio:53.79791304174515
住宅大樓(11層含以上有電梯)'s main_building_ratio null ratio is 0/405475
--------------------------------------------------
店面（店舖):
店面（店舖) mean ratio:73.83283676038607
店面（店舖)'s main_building_ratio null ratio is 0/7286
--------------------------------------------------
透天厝:
透天厝 mean ratio:100.0
透天厝's main_building_ratio null ratio is 0/22506
--------------------------------------------------
其他:
其他 mean ratio:49.82499999999999
其他's main_building_ratio

### 建立新表格把剩餘main_building_ratio補一補(就是其他拉)

In [20]:
tt_df = fillna_df.copy()

In [21]:
tt_df['main_building_ratio'].fillna(tt_df['main_building_ratio'].mean(), inplace=True)

In [22]:
# 確認一下
type_p_list = tt_df['property_type'].unique()
for type_p in type_p_list:
  print(f"{type_p}:")
  tmp = tt_df[tt_df['property_type'] == type_p ]['main_building_ratio']
  print(f"{type_p} mean ratio:{tmp.mean()}")
  print(f"{type_p}'s main_building_ratio null ratio is {tmp.isnull().sum()}/{len(tmp)}")
  print('-'*50)

公寓(5樓含以下無電梯):
公寓(5樓含以下無電梯) mean ratio:90.61000159863887
公寓(5樓含以下無電梯)'s main_building_ratio null ratio is 0/179542
--------------------------------------------------
華廈(10層含以下有電梯):
華廈(10層含以下有電梯) mean ratio:61.14878857359721
華廈(10層含以下有電梯)'s main_building_ratio null ratio is 0/128553
--------------------------------------------------
套房(1房(1廳)1衛):
套房(1房(1廳)1衛) mean ratio:60.6382584564407
套房(1房(1廳)1衛)'s main_building_ratio null ratio is 0/36627
--------------------------------------------------
住宅大樓(11層含以上有電梯):
住宅大樓(11層含以上有電梯) mean ratio:53.79791304174515
住宅大樓(11層含以上有電梯)'s main_building_ratio null ratio is 0/405475
--------------------------------------------------
店面（店舖):
店面（店舖) mean ratio:73.83283676038607
店面（店舖)'s main_building_ratio null ratio is 0/7286
--------------------------------------------------
透天厝:
透天厝 mean ratio:100.0
透天厝's main_building_ratio null ratio is 0/22506
--------------------------------------------------
其他:
其他 mean ratio:65.26971482052949
其他's main_building_ratio

## property_b_l_p_type

In [23]:
tt_df['property_type'].isnull().sum()

0

In [24]:
tt_df.drop(['parking_space_price', 'latitude', 'longitude','note', 'price_per_ping', 'total_price', 'cor', 'elevator_available', 'have_elevator'],axis=1, inplace=True)

In [ ]:
tt_df.info(max_cols=104)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777312 entries, 0 to 777311
Data columns (total 64 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   usage_category                        752817 non-null  object 
 1   address                               777312 non-null  object 
 2   property_type                         777312 non-null  object 
 3   community_name                        777312 non-null  int64  
 4   main_building_ratio                   777312 non-null  float64
 5   city                                  777312 non-null  object 
 6   transaction_date                      777312 non-null  object 
 7   building_age                          589815 non-null  float64
 8   number_of_land                        777312 non-null  int64  
 9   number_of_building                    777312 non-null  int64  
 10  number_of_parking_space               777312 non-null  int64  
 11  

In [25]:
# 房地(土地+建物)     0
# 房地(土地+建物)+車位   1

tt_df['property_b_l_p_type'] = np.where(tt_df['property_b_l_p_type'].isin(['房地(土地+建物)']), 0, 1)

In [26]:
tt_df['property_b_l_p_type'].unique()

array([0, 1])

## 要來填building age 和幾房廳

In [27]:
age_df = tt_df.copy()

In [28]:
age_list = age_df['property_type'].unique()

In [29]:
age_list = age_df['property_type'].unique()
for age_p in age_list:
  tmp = age_df[age_df['property_type'] == age_p ]['building_age']
  need_fillna_index = tmp[ tmp.isnull() ].index

  # 錯誤示範: 簡單來說只要是切片索引下就不能就用fillna直接改變dataframe
  # age_df.loc[need_fillna_index, 'main_building_ratio'].fillna(tmp.mean(), inplace=True)

  # 正確: 反正我們已經找到特定property_type下na的index了，在原先的data_clean使用loc只需要在指定欄位main_building_ratio，我們就可以直接進行填值(mean)
  mean_value = tmp.mean()
  age_df.loc[need_fillna_index, 'building_age'] = mean_value

In [30]:
age_df['building_age'].isnull().sum()

0

In [31]:
print('building_age')
for type_p in age_list:
  print(f"{type_p}:")
  tmp = age_df[age_df['property_type'] == type_p ]['building_age']
  print(f"{type_p} mean:{tmp.mean()}")
  print(f"{type_p}'s building_age null ratio is {tmp.isnull().sum()}/{len(tmp)}")
  print('-'*50)

building_age
公寓(5樓含以下無電梯):
公寓(5樓含以下無電梯) mean:37.16601928347498
公寓(5樓含以下無電梯)'s building_age null ratio is 0/179542
--------------------------------------------------
華廈(10層含以下有電梯):
華廈(10層含以下有電梯) mean:19.955749673813468
華廈(10層含以下有電梯)'s building_age null ratio is 0/128553
--------------------------------------------------
套房(1房(1廳)1衛):
套房(1房(1廳)1衛) mean:16.761150148472645
套房(1房(1廳)1衛)'s building_age null ratio is 0/36627
--------------------------------------------------
住宅大樓(11層含以上有電梯):
住宅大樓(11層含以上有電梯) mean:12.734924312859553
住宅大樓(11層含以上有電梯)'s building_age null ratio is 0/405475
--------------------------------------------------
店面（店舖):
店面（店舖) mean:23.931402964495
店面（店舖)'s building_age null ratio is 0/7286
--------------------------------------------------
透天厝:
透天厝 mean:30.802989519500606
透天厝's building_age null ratio is 0/22506
--------------------------------------------------
其他:
其他 mean:11.082527881040892
其他's building_age null ratio is 0/3408
----------------------------------------

In [ ]:
# age_df.to_csv('/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/EDA_fillna_building_age_ok.csv', encoding='utf-8', index=False) # '/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/xgboost_month_freq_dataset.csv'

## 填幾房幾廳
* 這邊是一個一個填，沒有寫迴圈，要重做的話要注意
* 基本上都已property type為分類依據，去填值

In [32]:
many_room_df = age_df.copy()

In [33]:
many_room_df.info(max_cols=104)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 783397 entries, 0 to 783396
Data columns (total 64 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   usage_category                        758618 non-null  object 
 1   address                               783397 non-null  object 
 2   property_type                         783397 non-null  object 
 3   community_name                        783397 non-null  int64  
 4   main_building_ratio                   783397 non-null  float64
 5   city                                  783397 non-null  object 
 6   transaction_date                      783397 non-null  object 
 7   building_age                          783397 non-null  float64
 8   number_of_land                        783397 non-null  int64  
 9   number_of_building                    783397 non-null  int64  
 10  number_of_parking_space               783397 non-null  int64  
 11  

In [34]:
def fillna_with_house_rooms(col_name):
  house_set_list = many_room_df['property_type'].unique()
  for age_p in house_set_list:
    tmp = many_room_df[many_room_df['property_type'] == age_p ][col_name]
    need_fillna_index = tmp[ tmp.isnull() ].index

    # 錯誤示範: 簡單來說只要是切片索引下就不能就用fillna直接改變dataframe
    # many_room_df.loc[need_fillna_index, 'main_building_ratio'].fillna(tmp.mean(), inplace=True)

    # 正確反正我們已經找到特定property_type下na的index了，在原先的data_clean使用loc只需要在指定欄位main_building_ratio，我們就可以直接進行填值(mean)
    mean_value = tmp.mean().round(0)
    many_room_df.loc[need_fillna_index, col_name] = mean_value

  type_p_list = many_room_df['property_type'].unique()
  print('col_name')
  print('='*50)
  for type_p in type_p_list:
    print(f"{type_p}:")
    tmp = many_room_df[many_room_df['property_type'] == type_p ][col_name]
    print(f"{type_p} mean:{tmp.mean()}")
    print(f"{type_p}'s col_name null ratio is {tmp.isnull().sum()}/{len(tmp)}")
    print('-'*50)

In [35]:
rooms_list = many_room_df.columns[17:23]
for col_name in rooms_list:
  fillna_with_house_rooms(col_name)

col_name
公寓(5樓含以下無電梯):
公寓(5樓含以下無電梯) mean:4.502489668155641
公寓(5樓含以下無電梯)'s col_name null ratio is 0/179542
--------------------------------------------------
華廈(10層含以下有電梯):
華廈(10層含以下有電梯) mean:7.666363289849323
華廈(10層含以下有電梯)'s col_name null ratio is 0/128553
--------------------------------------------------
套房(1房(1廳)1衛):
套房(1房(1廳)1衛) mean:13.539028585469735
套房(1房(1廳)1衛)'s col_name null ratio is 0/36627
--------------------------------------------------
住宅大樓(11層含以上有電梯):
住宅大樓(11層含以上有電梯) mean:16.98944694494112
住宅大樓(11層含以上有電梯)'s col_name null ratio is 0/405475
--------------------------------------------------
店面（店舖):
店面（店舖) mean:9.094015920944276
店面（店舖)'s col_name null ratio is 0/7286
--------------------------------------------------
透天厝:
透天厝 mean:2.691371189904914
透天厝's col_name null ratio is 0/22506
--------------------------------------------------
其他:
其他 mean:14.511737089201878
其他's col_name null ratio is 0/3408
--------------------------------------------------
col_name
公寓(5樓含以下無電梯):

In [36]:
type_p_list = many_room_df['property_type'].unique()
print('transaction_floors')
print('='*50)
for type_p in type_p_list:
  print(f"{type_p}:")
  tmp = many_room_df[many_room_df['property_type'] == type_p ]['transaction_floors']
  print(f"{type_p} mean:{tmp.mean()}")
  print(f"{type_p}'s transaction_floors null ratio is {tmp.isnull().sum()}/{len(tmp)}")
  print('-'*50)

transaction_floors
公寓(5樓含以下無電梯):
公寓(5樓含以下無電梯) mean:1.028333203373027
公寓(5樓含以下無電梯)'s transaction_floors null ratio is 0/179542
--------------------------------------------------
華廈(10層含以下有電梯):
華廈(10層含以下有電梯) mean:1.022130949880594
華廈(10層含以下有電梯)'s transaction_floors null ratio is 0/128553
--------------------------------------------------
套房(1房(1廳)1衛):
套房(1房(1廳)1衛) mean:1.019684931880853
套房(1房(1廳)1衛)'s transaction_floors null ratio is 0/36627
--------------------------------------------------
住宅大樓(11層含以上有電梯):
住宅大樓(11層含以上有電梯) mean:1.0123435476909797
住宅大樓(11層含以上有電梯)'s transaction_floors null ratio is 0/405475
--------------------------------------------------
店面（店舖):
店面（店舖) mean:1.1169365907219324
店面（店舖)'s transaction_floors null ratio is 0/7286
--------------------------------------------------
透天厝:
透天厝 mean:2.691371189904914
透天厝's transaction_floors null ratio is 0/22506
--------------------------------------------------
其他:
其他 mean:1.2514671361502347
其他's transaction_floors null ratio is

In [37]:
house_set_list = many_room_df['property_type'].unique()
for age_p in house_set_list:
  tmp = many_room_df[many_room_df['property_type'] == age_p ]['transaction_floors']
  need_fillna_index = tmp[ tmp.isnull() ].index

  # 錯誤示範: 簡單來說只要是切片索引下就不能就用fillna直接改變dataframe
  # many_room_df.loc[need_fillna_index, 'main_building_ratio'].fillna(tmp.mean(), inplace=True)

  # 正確反正我們已經找到特定property_type下na的index了，在原先的data_clean使用loc只需要在指定欄位main_building_ratio，我們就可以直接進行填值(mean)
  mean_value = tmp.mean().round(0)
  many_room_df.loc[need_fillna_index, 'transaction_floors'] = mean_value

In [38]:
many_room_df.info(max_cols=104)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 783397 entries, 0 to 783396
Data columns (total 64 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   usage_category                        758618 non-null  object 
 1   address                               783397 non-null  object 
 2   property_type                         783397 non-null  object 
 3   community_name                        783397 non-null  int64  
 4   main_building_ratio                   783397 non-null  float64
 5   city                                  783397 non-null  object 
 6   transaction_date                      783397 non-null  object 
 7   building_age                          783397 non-null  float64
 8   number_of_land                        783397 non-null  int64  
 9   number_of_building                    783397 non-null  int64  
 10  number_of_parking_space               783397 non-null  int64  
 11  

## usage category

In [39]:
many_room_df.isnull().sum().sum()

50970

In [40]:
uu_df = many_room_df.copy()

In [41]:
uu_df['usage_category'].unique()

array(['住', '商', '6', '工', '其他', '7', '農', nan], dtype=object)

In [42]:
# 把null值跟"6","7"，都改成"其他"
uu_df['usage_category'].fillna(value="其他", inplace=True)

for index, value in enumerate(uu_df['usage_category']):
    if value == "6":
        uu_df['usage_category'].iloc[index] = "其他"
    if value == "7":
        uu_df['usage_category'].iloc[index] = "其他"

In [43]:
uu_df['usage_category'].isnull().sum()

0

In [44]:
uu_df.isnull().sum().sum()

26191

In [45]:
uu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 783397 entries, 0 to 783396
Data columns (total 64 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   usage_category                        783397 non-null  object 
 1   address                               783397 non-null  object 
 2   property_type                         783397 non-null  object 
 3   community_name                        783397 non-null  int64  
 4   main_building_ratio                   783397 non-null  float64
 5   city                                  783397 non-null  object 
 6   transaction_date                      783397 non-null  object 
 7   building_age                          783397 non-null  float64
 8   number_of_land                        783397 non-null  int64  
 9   number_of_building                    783397 non-null  int64  
 10  number_of_parking_space               783397 non-null  int64  
 11  

In [ ]:
# uu_df.to_csv('/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/EDA_fillna_ok_not_economic_select.csv', encoding='utf-8', index=False) # '/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/xgboost_month_freq_dataset.csv'

# 更改成跟591資料集靠攏

In [55]:
need_df = uu_df.copy()

In [47]:
need_df['number_of_parking_space'].value_counts()

number_of_parking_space
0     431441
1     303978
2      42102
3       4790
4        786
5        148
6         70
7         31
9         12
8         11
10         8
12         6
14         3
15         2
11         2
60         2
13         1
22         1
18         1
23         1
16         1
Name: count, dtype: int64

In [56]:
need_df['number_of_parking_space'] = np.where(need_df['number_of_parking_space'].isin([0]), 0, 1)

In [57]:
need_df['has_parking_space'] = need_df['number_of_parking_space']

In [58]:
need_df.drop('number_of_parking_space', axis=1, inplace=True)

In [51]:
need_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 783397 entries, 0 to 783396
Data columns (total 64 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   usage_category                        783397 non-null  object 
 1   address                               783397 non-null  object 
 2   property_type                         783397 non-null  object 
 3   community_name                        783397 non-null  int64  
 4   main_building_ratio                   783397 non-null  float64
 5   city                                  783397 non-null  object 
 6   transaction_date                      783397 non-null  object 
 7   building_age                          783397 non-null  float64
 8   number_of_land                        783397 non-null  int64  
 9   number_of_building                    783397 non-null  int64  
 10  management_org_available              783397 non-null  int64  
 11  

In [59]:
usage_df = need_df[ (need_df['detailed_usage_classification'] == '住家用') | (need_df['detailed_usage_classification'] == '住商用') ]

In [61]:
usage_df.drop('detailed_usage_classification', axis=1,inplace=True)

<ipython-input-61-a7c86e2b320d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usage_df.drop('detailed_usage_classification', axis=1,inplace=True)


In [60]:
usage_df['usage_category'] = usage_df['detailed_usage_classification']

<ipython-input-60-1d98350655ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usage_df['usage_category'] = usage_df['detailed_usage_classification']


In [62]:
usage_df = usage_df[usage_df['per_ping_calculation_method'] != '總面積-車位總面積 剩餘面積小於1平方公尺 ,不計算單價']

In [63]:
usage_df.drop('date',axis=1,inplace=True)

In [64]:
usage_df['elevator_available'] = usage_df['have_elevator_f']

In [65]:
usage_df['property_type'].unique()

array(['公寓(5樓含以下無電梯)', '華廈(10層含以下有電梯)', '套房(1房(1廳)1衛)', '店面（店舖)', '透天厝',
       '住宅大樓(11層含以上有電梯)', '其他'], dtype=object)

In [66]:
usage_df['property_type'].replace({'公寓(5樓含以下無電梯)':'公寓', '華廈(10層含以下有電梯)': '電梯大樓', '住宅大樓(11層含以上有電梯)': '電梯大樓'}, inplace=True)

In [67]:
usage_df['property_type'].value_counts()

property_type
電梯大樓            481399
公寓              168453
套房(1房(1廳)1衛)     28723
透天厝              19882
店面（店舖)            2186
其他                 823
Name: count, dtype: int64

In [68]:
usage_df_2 = usage_df[ (usage_df['property_type'] == '公寓') | (usage_df['property_type'] == '電梯大樓') | (usage_df['property_type'] == '透天厝')]

In [69]:
usage_df_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 669734 entries, 0 to 783396
Data columns (total 63 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   usage_category                        669734 non-null  object 
 1   address                               669734 non-null  object 
 2   property_type                         669734 non-null  object 
 3   community_name                        669734 non-null  int64  
 4   main_building_ratio                   669734 non-null  float64
 5   city                                  669734 non-null  object 
 6   transaction_date                      669734 non-null  object 
 7   building_age                          669734 non-null  float64
 8   number_of_land                        669734 non-null  int64  
 9   number_of_building                    669734 non-null  int64  
 10  management_org_available              669734 non-null  int64  
 11  per_p

In [70]:
usage_df['usage_category'].unique()

array(['住家用', '住商用'], dtype=object)

In [71]:
usage_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 701466 entries, 0 to 783396
Data columns (total 63 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   usage_category                        701466 non-null  object 
 1   address                               701466 non-null  object 
 2   property_type                         701466 non-null  object 
 3   community_name                        701466 non-null  int64  
 4   main_building_ratio                   701466 non-null  float64
 5   city                                  701466 non-null  object 
 6   transaction_date                      701466 non-null  object 
 7   building_age                          701466 non-null  float64
 8   number_of_land                        701466 non-null  int64  
 9   number_of_building                    701466 non-null  int64  
 10  management_org_available              701466 non-null  int64  
 11  per_p

In [ ]:
usage_df_2.to_csv('/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/main_data_match_591_before_one_hot.csv', encoding='utf-8', index=False)

In [72]:
district_match_dict = {
    "A15": "士林區",
    "A09": "大同區",
    "A02": "大安區",
    "A10": "中山區",
    "A03": "中正區",
    "A14": "內湖區",
    "A11": "文山區",
    "A16": "北投區",
    "A01": "松山區",
    "A17": "信義區",
    "A13": "南港區",
    "A05": "萬華區",
    "F32": "八里區",
    "F30": "三芝區",
    "F05": "三重區",
    "F15": "三峽區",
    "F19": "土城區",
    "F18": "中和區",
    "F03": "五股區",
    "F22": "平溪區",
    "F33": "永和區",
    "F31": "石門區",
    "F08": "石碇區",
    "F28": "汐止區",
    "F10": "坪林區",
    "F02": "林口區",
    "F14": "板橋區",
    "F25": "金山區",
    "F06": "泰山區",
    "F11": "烏來區",
    "F24": "貢寮區",
    "F27": "淡水區",
    "F09": "深坑區",
    "F07": "新店區",
    "F01": "新莊區",
    "F21": "瑞芳區",
    "F26": "萬里區",
    "F17": "樹林區",
    "F23": "雙溪區",
    "F04": "蘆洲區",
    "F16": "鶯歌區"
}


In [73]:
usage_df_2['district'].replace(district_match_dict, inplace=True)

<ipython-input-73-55ad7f01e1dd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usage_df_2['district'].replace(district_match_dict, inplace=True)


In [74]:
usage_df_2['city'].unique()

array(['A', 'F'], dtype=object)

In [75]:
usage_df_2['city'].replace({'A':'台北市', 'F': '新北市'}, inplace=True)

<ipython-input-75-f07717d095e2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usage_df_2['city'].replace({'A':'台北市', 'F': '新北市'}, inplace=True)


In [76]:
usage_df_2.to_csv('/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/final_full_main_data_before_one_hot_can_do_EDA.csv', encoding='utf-8', index=False)

# 編碼
* 最好先處理下面的usage

In [77]:
another_df = usage_df_2.copy()

In [78]:
# 會自己把原先欄位drop掉
another_df = pd.get_dummies(another_df, columns=['property_type'], prefix='property_type_')
another_df = pd.get_dummies(another_df, columns=['city'], prefix='city_')
# another_df = pd.get_dummies(another_df, columns=['property_b_l_p_type'], prefix='property_b_l_p_type_')
another_df = pd.get_dummies(another_df, columns=['district'], prefix='district_')
another_df = pd.get_dummies(another_df, columns=['usage_category'], prefix='usage_category_')

In [79]:
another_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 669734 entries, 0 to 783396
Columns: 107 entries, address to usage_category__住家用
dtypes: bool(48), float64(47), int64(8), object(4)
memory usage: 337.2+ MB


In [80]:
another_df.iloc[:,49:] = another_df.iloc[:,49:].astype(int)

In [81]:
another_df.isnull().sum().sum()

15894

In [82]:
another_df.info(max_cols=131)

<class 'pandas.core.frame.DataFrame'>
Index: 669734 entries, 0 to 783396
Data columns (total 107 columns):
 #    Column                                Non-Null Count   Dtype  
---   ------                                --------------   -----  
 0    address                               669734 non-null  object 
 1    community_name                        669734 non-null  int64  
 2    main_building_ratio                   669734 non-null  float64
 3    transaction_date                      669734 non-null  object 
 4    building_age                          669734 non-null  float64
 5    number_of_land                        669734 non-null  int64  
 6    number_of_building                    669734 non-null  int64  
 7    management_org_available              669734 non-null  int64  
 8    per_ping_calculation_method           669734 non-null  object 
 9    total_area_ping                       669734 non-null  float64
 10   property_b_l_p_type                   669734 non-null  int6

In [83]:
another_df.drop('have_elevator_f', axis=1, inplace=True)

In [84]:
another_df.to_csv('/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/final_full_main_data_after_one_hot_chinese.csv', encoding='utf-8', index=False) # '/content/my_drive/MyDrive/BDSE34期末報告_房價/all_data/xgboost_month_freq_dataset.csv'